In [ ]:
import cv2
import torch
import torch.nn as nn
import torchvision.models as models
import time
from collections import deque
import threading
import queue
from gtts import gTTS
import pygame
import io
import tempfile
import os
import math
import numpy as np

# Your existing model class (unchanged)
class SIBIModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.frame_encoder = torch.nn.Sequential(*list(models.convnext_tiny(pretrained=True).children())[:-1])
        self.frame_decoder = nn.Linear(768, 64, dtype=torch.float64)
        self.l1 = nn.LSTM(64, 64, batch_first=True, dtype=torch.float64)
        self.out = nn.Linear(64, 20, dtype=torch.float64)
        self.dropout = nn.Dropout(p=0.2)
    
    def forward(self, x):
        input_size = x.shape
        x = torch.squeeze(x, 0)
        encoded_x = self.frame_encoder(x.float())
        encoded_x = torch.squeeze(encoded_x).double()
        decoded_x = self.frame_decoder(encoded_x)
        decoded_x = torch.unsqueeze(decoded_x, 0)
        out = self.l1(decoded_x)
        hidden_out = out[0][:, -1, :]
        output = self.out(hidden_out)
        return output

class RealTimeSIBIInference:
    def __init__(self, model_path, class_names, device='cuda',
                 sequence_length=30, confidence_threshold=0.8,
                 cooldown_frames=15, class_specific_thresholds=None,
                 auto_speech=True):
        """
        Initialize real-time SIBI inference with enhanced professional UI
        """
        self.device = torch.device(device if torch.cuda.is_available() else "cpu")
        self.class_names = class_names
        self.sequence_length = sequence_length
        self.confidence_threshold = confidence_threshold
        self.cooldown_frames = cooldown_frames
        self.class_specific_thresholds = class_specific_thresholds or {}
        self.auto_speech = auto_speech
        
        # Enhanced UI Colors (Professional Dark Theme)
        self.colors = {
            'bg_dark': (20, 25, 35),           # Dark navy background
            'bg_medium': (35, 42, 58),         # Medium blue-gray
            'bg_light': (50, 60, 80),          # Light blue-gray
            'primary': (0, 150, 255),          # Bright blue
            'secondary': (100, 180, 255),      # Light blue
            'success': (50, 205, 50),          # Green
            'warning': (255, 165, 0),          # Orange
            'error': (220, 53, 69),            # Red
            'text_primary': (255, 255, 255),   # White
            'text_secondary': (180, 190, 210), # Light gray-blue
            'text_muted': (120, 130, 150),     # Muted gray
            'accent': (138, 43, 226),          # Purple
            'tts_active': (255, 215, 0),       # Gold
            'record_active': (255, 69, 58),    # Vibrant red
        }
        
        print(f"Enhanced Professional UI Loaded")
        print(f"Loading model on device: {self.device}")
        print(f"Number of classes: {len(self.class_names)}")
        print(f"Default confidence threshold: {self.confidence_threshold}")
        print(f"Auto-speech enabled: {self.auto_speech}")
        if self.class_specific_thresholds:
            print(f"Class-specific thresholds: {self.class_specific_thresholds}")
        print(f"Classes: {self.class_names}")
        
        # Load model
        self.model = SIBIModel()
        state_dict = torch.load(model_path, map_location=self.device)
        self.model.load_state_dict(state_dict)
        self.model.to(self.device)
        self.model.eval()
        
        # Frame buffer
        self.frame_buffer = deque(maxlen=sequence_length)
        self.prediction_queue = queue.Queue()
        
        # State variables
        self.is_recording = False
        self.cooldown_counter = 0
        self.detected_words = []
        self.current_prediction = ""
        self.current_confidence = 0.0
        self.last_prediction_time = 0
        
        # Real-time prediction info
        self.realtime_prediction = ""
        self.realtime_confidence = 0.0
        
        # FPS tracking
        self.fps_counter = deque(maxlen=30)
        self.last_frame_time = time.time()
        self.current_fps = 0.0
        
        # FPS Statistics
        self.session_fps_history = []
        self.recording_start_time = None
        self.recording_end_time = None
        self.fps_samples_during_recording = []
        
        # Duplicate Prevention
        self.last_accepted_word = None
        
        # TTS Queue Management
        self.tts_queue = queue.Queue()
        self.is_speaking = False
        
        # Simple notification system (removed - no notifications)
        # self.notification_queue = deque(maxlen=5)
        
        # Initialize TTS
        try:
            pygame.mixer.init(frequency=22050, size=-16, channels=2, buffer=512)
            self.tts_available = True
            print("Professional TTS System initialized successfully")
            self.start_tts_worker()
        except Exception as e:
            print(f"Warning: Could not initialize TTS: {e}")
            self.tts_available = False
        
        # Camera
        self.cap = None
    
    def add_notification(self, message, type="info", duration=3):
        """Notifications removed - no longer used"""
        pass
    
    def start_tts_worker(self):
        """Start the TTS worker thread"""
        def tts_worker():
            while True:
                try:
                    text = self.tts_queue.get(timeout=1)
                    if text is None:
                        break
                    
                    self.is_speaking = True
                    print(f"Speaking: {text}")
                    
                    tts = gTTS(text=text, lang='id', slow=False)
                    
                    with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as tmp_file:
                        tts.save(tmp_file.name)
                        tmp_filename = tmp_file.name
                    
                    pygame.mixer.music.load(tmp_filename)
                    pygame.mixer.music.play()
                    
                    while pygame.mixer.music.get_busy():
                        time.sleep(0.1)
                    
                    try:
                        os.unlink(tmp_filename)
                    except:
                        pass
                    
                    self.is_speaking = False
                    self.tts_queue.task_done()
                    
                except queue.Empty:
                    continue
                except Exception as e:
                    print(f"TTS Error: {e}")
                    self.is_speaking = False
                    self.tts_queue.task_done()
        
        self.tts_thread = threading.Thread(target=tts_worker, daemon=True)
        self.tts_thread.start()
    
    def speak_text(self, text, immediate=False):
        """Speak text with professional feedback"""
        if not self.tts_available or not text.strip():
            return
        
        if immediate:
            def immediate_tts():
                try:
                    print(f"Manual speech: {text}")
                    self.add_notification(f"Speaking: {text}", "tts", 2)
                    tts = gTTS(text=text, lang='id', slow=False)
                    
                    with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as tmp_file:
                        tts.save(tmp_file.name)
                        tmp_filename = tmp_file.name
                    
                    sound = pygame.mixer.Sound(tmp_filename)
                    sound.play()
                    
                    while pygame.mixer.get_busy():
                        time.sleep(0.1)
                    
                    try:
                        os.unlink(tmp_filename)
                    except:
                        pass
                        
                except Exception as e:
                    print(f"TTS Error: {e}")
                    self.add_notification("TTS Error - Check internet", "error", 3)
            
            immediate_thread = threading.Thread(target=immediate_tts, daemon=True)
            immediate_thread.start()
        else:
            if not self.is_speaking and self.tts_queue.empty():
                self.tts_queue.put(text)
    
    def update_fps(self):
        """Update FPS calculation"""
        current_time = time.time()
        frame_time = current_time - self.last_frame_time
        self.fps_counter.append(frame_time)
        self.last_frame_time = current_time
        
        if len(self.fps_counter) > 0:
            avg_frame_time = sum(self.fps_counter) / len(self.fps_counter)
            self.current_fps = 1.0 / avg_frame_time if avg_frame_time > 0 else 0.0
        
        if self.is_recording:
            self.fps_samples_during_recording.append(self.current_fps)

    def start_recording_session(self):
        """Start recording with professional feedback"""
        self.is_recording = True
        self.recording_start_time = time.time()
        self.fps_samples_during_recording.clear()
        self.frame_buffer.clear()
        self.realtime_prediction = ""
        self.realtime_confidence = 0.0
        print("Recording started - perform sign language gestures!")

    def stop_recording_session(self):
        """Stop recording with statistics"""
        self.is_recording = False
        self.recording_end_time = time.time()
        
        if self.fps_samples_during_recording:
            session_avg_fps = sum(self.fps_samples_during_recording) / len(self.fps_samples_during_recording)
            session_duration = self.recording_end_time - self.recording_start_time
            self.session_fps_history.extend(self.fps_samples_during_recording)
            
            print(f"Recording Session Statistics:")
            print(f"   Duration: {session_duration:.1f} seconds")
            print(f"   Average FPS: {session_avg_fps:.1f}")
        else:
            print("Recording stopped")

    def get_overall_fps_stats(self):
        """Get overall FPS statistics"""
        if not self.session_fps_history:
            return None
        
        return {
            'average': sum(self.session_fps_history) / len(self.session_fps_history),
            'min': min(self.session_fps_history),
            'max': max(self.session_fps_history),
            'total_frames': len(self.session_fps_history)
        }
        
    def get_threshold_for_class(self, class_name):
        """Get confidence threshold for specific class"""
        return self.class_specific_thresholds.get(class_name, self.confidence_threshold)
        
    def preprocess_frame(self, frame):
        """Preprocess frame for model input"""
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_resized = cv2.resize(frame_rgb, (224, 224))
        frame_tensor = torch.from_numpy(frame_resized).permute(2, 0, 1).float()
        frame_tensor = frame_tensor / 255.0
        
        mean = torch.tensor([0.485, 0.456, 0.406])
        std = torch.tensor([0.229, 0.224, 0.225])
        frame_tensor = (frame_tensor - mean.view(3, 1, 1)) / std.view(3, 1, 1)
        
        return frame_tensor.double()
    
    def predict_sequence(self):
        """Run inference on current frame sequence"""
        if len(self.frame_buffer) < self.sequence_length:
            return None, 0.0
        
        frames = torch.stack(list(self.frame_buffer))
        frames = frames.unsqueeze(0).to(self.device)
        
        with torch.no_grad():
            logits = self.model(frames)
            probabilities = torch.softmax(logits, dim=1)
            confidence, predicted_idx = torch.max(probabilities, dim=1)
            
            predicted_class = self.class_names[predicted_idx.item()]
            confidence_score = confidence.item()
            
        return predicted_class, confidence_score
    
    def process_frame(self, frame):
        """Process frame with enhanced feedback"""
        processed_frame = self.preprocess_frame(frame)
        self.frame_buffer.append(processed_frame)
        
        if self.cooldown_counter > 0:
            self.cooldown_counter -= 1
            return
        
        if len(self.frame_buffer) == self.sequence_length:
            prediction, confidence = self.predict_sequence()
            
            if prediction:
                self.realtime_prediction = prediction
                self.realtime_confidence = confidence
            
            required_threshold = self.get_threshold_for_class(prediction) if prediction else self.confidence_threshold
            
            if prediction and confidence >= required_threshold:
                if prediction == self.last_accepted_word:
                    print(f"DUPLICATE REJECTED: {prediction}")
                    return
                
                self.current_prediction = prediction
                self.current_confidence = confidence
                self.detected_words.append(prediction)
                self.last_accepted_word = prediction
                self.cooldown_counter = self.cooldown_frames
                self.last_prediction_time = time.time()
                

                print(f"ACCEPTED: {prediction} (confidence: {confidence:.3f})")
                
                if self.auto_speech:
                    self.speak_text(prediction, immediate=False)
    
    def draw_rounded_rect(self, img, pt1, pt2, color, thickness=-1, radius=10):
        """Draw rounded rectangle for modern UI"""
        x1, y1 = int(pt1[0]), int(pt1[1])
        x2, y2 = int(pt2[0]), int(pt2[1])
        
        # Ensure proper coordinates
        if x1 > x2:
            x1, x2 = x2, x1
        if y1 > y2:
            y1, y2 = y2, y1
            
        # Ensure radius doesn't exceed rectangle dimensions
        radius = min(radius, (x2 - x1) // 2, (y2 - y1) // 2)
        if radius <= 0:
            # Fallback to regular rectangle if radius is too small
            if thickness == -1:
                cv2.rectangle(img, (x1, y1), (x2, y2), color, -1)
            else:
                cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness)
            return
            
        # Create mask for rounded corners
        mask = np.zeros(img.shape[:2], dtype=np.uint8)
        
        # Draw rounded rectangle on mask - ensure all coordinates are integers
        cv2.rectangle(mask, (x1 + radius, y1), (x2 - radius, y2), 255, -1)
        cv2.rectangle(mask, (x1, y1 + radius), (x2, y2 - radius), 255, -1)
        cv2.circle(mask, (x1 + radius, y1 + radius), radius, 255, -1)
        cv2.circle(mask, (x2 - radius, y1 + radius), radius, 255, -1)
        cv2.circle(mask, (x1 + radius, y2 - radius), radius, 255, -1)
        cv2.circle(mask, (x2 - radius, y2 - radius), radius, 255, -1)
        
        # Apply color to masked area
        if thickness == -1:
            img[mask == 255] = color
        else:
            # For border, create inner mask and subtract
            inner_mask = np.zeros(img.shape[:2], dtype=np.uint8)
            inner_radius = max(0, radius - thickness)
            inner_x1, inner_y1 = x1 + thickness, y1 + thickness
            inner_x2, inner_y2 = x2 - thickness, y2 - thickness
            
            if inner_x2 > inner_x1 and inner_y2 > inner_y1 and inner_radius > 0:
                cv2.rectangle(inner_mask, (inner_x1 + inner_radius, inner_y1), 
                             (inner_x2 - inner_radius, inner_y2), 255, -1)
                cv2.rectangle(inner_mask, (inner_x1, inner_y1 + inner_radius), 
                             (inner_x2, inner_y2 - inner_radius), 255, -1)
                cv2.circle(inner_mask, (inner_x1 + inner_radius, inner_y1 + inner_radius), 
                          inner_radius, 255, -1)
                cv2.circle(inner_mask, (inner_x2 - inner_radius, inner_y1 + inner_radius), 
                          inner_radius, 255, -1)
                cv2.circle(inner_mask, (inner_x1 + inner_radius, inner_y2 - inner_radius), 
                          inner_radius, 255, -1)
                cv2.circle(inner_mask, (inner_x2 - inner_radius, inner_y2 - inner_radius), 
                          inner_radius, 255, -1)
            
            border_mask = cv2.bitwise_xor(mask, inner_mask)
            img[border_mask == 255] = color
    
    def draw_progress_bar(self, img, x, y, width, height, progress, bg_color, fill_color, radius=5):
        """Draw modern progress bar"""
        # Ensure all coordinates are integers
        x, y, width, height, radius = int(x), int(y), int(width), int(height), int(radius)
        
        # Background
        self.draw_rounded_rect(img, (x, y), (x + width, y + height), bg_color, -1, radius)
        
        # Fill
        if progress > 0:
            fill_width = int(width * min(progress, 1.0))
            if fill_width > 0:
                self.draw_rounded_rect(img, (x, y), (x + fill_width, y + height), fill_color, -1, radius)
    
    def draw_modern_button(self, img, x, y, width, height, text, active=False, color_scheme='primary'):
        """Draw modern button with hover effects"""
        # Ensure all coordinates are integers
        x, y, width, height = int(x), int(y), int(width), int(height)
        
        if color_scheme == 'primary':
            bg_color = self.colors['primary'] if active else self.colors['bg_medium']
            text_color = self.colors['text_primary']
        elif color_scheme == 'success':
            bg_color = self.colors['success'] if active else self.colors['bg_medium']
            text_color = self.colors['text_primary']
        elif color_scheme == 'warning':
            bg_color = self.colors['warning'] if active else self.colors['bg_medium']
            text_color = self.colors['text_primary']
        elif color_scheme == 'error':
            bg_color = self.colors['error'] if active else self.colors['bg_medium']
            text_color = self.colors['text_primary']
        elif color_scheme == 'accent':
            bg_color = self.colors['accent'] if active else self.colors['bg_medium']
            text_color = self.colors['text_primary']
        else:
            bg_color = self.colors['bg_medium']
            text_color = self.colors['text_secondary']
        
        # Draw button background
        self.draw_rounded_rect(img, (x, y), (x + width, y + height), bg_color, -1, 8)
        
        # Add subtle border
        border_color = self.colors['text_muted'] if not active else self.colors['text_secondary']
        self.draw_rounded_rect(img, (x, y), (x + width, y + height), border_color, 1, 8)
        
        # Center text
        text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]
        text_x = x + (width - text_size[0]) // 2
        text_y = y + (height + text_size[1]) // 2
        
        cv2.putText(img, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1)
    
    def draw_notification_panel(self, frame, width, height):
        """Notifications completely removed"""
        pass
    
    def draw_enhanced_info_panel(self, frame, width, height):
        """Draw the enhanced professional info panel (no animations)"""
        panel_width = 420
        panel_height = height
        panel_x = width - panel_width
        
        # Modern dark background
        overlay = frame.copy()
        self.draw_rounded_rect(overlay, (panel_x, 0), (width, height), self.colors['bg_dark'], -1, 0)
        frame = cv2.addWeighted(frame, 0.3, overlay, 0.7, 0)
        
        # Modern border
        cv2.line(frame, (panel_x, 0), (panel_x, height), self.colors['primary'], 3)
        
        y_pos = 40
        line_height = 30
        
        # Professional Title
        title_bg_width = panel_width - 20
        self.draw_rounded_rect(frame, (panel_x + 10, y_pos - 15), 
                              (panel_x + 10 + title_bg_width, y_pos + 20), 
                              self.colors['bg_medium'], -1, 8)
        
        cv2.putText(frame, "SIBI PROFESSIONAL", (int(panel_x + 20), int(y_pos)), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, self.colors['text_primary'], 2)
        cv2.putText(frame, "Real-time Detection", (int(panel_x + 20), int(y_pos + 20)), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, self.colors['text_secondary'], 1)
        
        y_pos += line_height * 2.5
        
        # FPS Display with color coding (no animations)
        fps_color = self.colors['success'] if self.current_fps > 20 else \
                   self.colors['warning'] if self.current_fps > 15 else self.colors['error']
        
        # FPS Card
        card_height = 45
        self.draw_rounded_rect(frame, (panel_x + 15, y_pos - 10), 
                              (panel_x + panel_width - 15, y_pos + card_height - 10), 
                              self.colors['bg_medium'], -1, 8)
        
        cv2.putText(frame, f"FPS: {self.current_fps:.1f}", (int(panel_x + 25), int(y_pos + 8)), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, fps_color, 2)
        
        y_pos += line_height * 2
        
        # Status Section (no pulsing effects)
        status_card_height = 80
        self.draw_rounded_rect(frame, (panel_x + 15, y_pos - 10), 
                              (panel_x + panel_width - 15, y_pos + status_card_height), 
                              self.colors['bg_light'], -1, 8)
        
        # Recording Status (no pulsing effect)
        if self.is_recording:
            record_color = self.colors['record_active']
            status_text = "RECORDING"
        else:
            record_color = self.colors['text_muted']
            status_text = "STANDBY"
        
        cv2.putText(frame, status_text, (int(panel_x + 25), int(y_pos + 15)), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, record_color, 2)
        
        # Auto-speech indicator
        auto_color = self.colors['success'] if self.auto_speech else self.colors['text_muted']
        cv2.putText(frame, f"Auto-Speech: {'ON' if self.auto_speech else 'OFF'}", 
                   (int(panel_x + 25), int(y_pos + 35)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, auto_color, 1)
        
        # Speaking indicator (no animation)
        if self.is_speaking:
            speaking_color = self.colors['tts_active']
            cv2.putText(frame, "SPEAKING", (int(panel_x + 25), int(y_pos + 55)), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, speaking_color, 1)
        
        y_pos += status_card_height + 20
        
        # Buffer Status
        buffer_ratio = len(self.frame_buffer) / self.sequence_length
        cv2.putText(frame, f"Buffer: {len(self.frame_buffer)}/{self.sequence_length}", 
                   (int(panel_x + 25), int(y_pos)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, 
                   self.colors['text_secondary'], 1)
        
        # Progress bar
        progress_width = panel_width - 50
        progress_height = 8
        y_pos += 15
        
        self.draw_progress_bar(frame, panel_x + 25, y_pos, progress_width, progress_height, 
                              buffer_ratio, self.colors['bg_medium'], self.colors['primary'], 4)
        
        y_pos += 25
        
        # Cooldown indicator
        if self.cooldown_counter > 0:
            cooldown_ratio = self.cooldown_counter / self.cooldown_frames
            cv2.putText(frame, f"Cooldown: {self.cooldown_counter}", (int(panel_x + 25), int(y_pos)), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, self.colors['warning'], 1)
            y_pos += 15
            self.draw_progress_bar(frame, panel_x + 25, y_pos, progress_width, progress_height, 
                                  cooldown_ratio, self.colors['bg_medium'], self.colors['warning'], 4)
            y_pos += 20
        
        y_pos += 15
        
        # Real-time Prediction Card
        prediction_card_height = 120
        self.draw_rounded_rect(frame, (panel_x + 15, y_pos), 
                              (panel_x + panel_width - 15, y_pos + prediction_card_height), 
                              self.colors['bg_medium'], -1, 8)
        
        cv2.putText(frame, "REAL-TIME PREDICTION", (int(panel_x + 25), int(y_pos + 20)), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, self.colors['text_secondary'], 1)
        
        if self.realtime_prediction:
            required_threshold = self.get_threshold_for_class(self.realtime_prediction)
            is_duplicate = self.realtime_prediction == self.last_accepted_word
            
            # Color coding for prediction status
            if is_duplicate:
                pred_color = self.colors['text_muted']
                status_text = "DUPLICATE"
            elif self.realtime_confidence >= required_threshold:
                pred_color = self.colors['success']
                status_text = "VALID"
            else:
                pred_color = self.colors['warning']
                status_text = "LOW CONF"
            
            # Large prediction text
            cv2.putText(frame, self.realtime_prediction, (int(panel_x + 25), int(y_pos + 50)), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1.0, pred_color, 2)
            
            # Confidence and status
            cv2.putText(frame, f"{self.realtime_confidence:.3f} | {status_text}", 
                       (int(panel_x + 25), int(y_pos + 70)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, pred_color, 1)
            
            # Threshold bar
            conf_bar_width = panel_width - 50
            conf_bar_height = 6
            y_bar = y_pos + 85
            
            # Background bar
            self.draw_progress_bar(frame, panel_x + 25, y_bar, conf_bar_width, conf_bar_height, 
                                  1.0, self.colors['bg_dark'], self.colors['bg_dark'], 3)
            
            # Confidence bar
            conf_progress = min(self.realtime_confidence, 1.0)
            self.draw_progress_bar(frame, panel_x + 25, y_bar, conf_bar_width, conf_bar_height, 
                                  conf_progress, self.colors['bg_dark'], pred_color, 3)
            
            # Threshold marker
            threshold_pos = int(conf_bar_width * required_threshold)
            cv2.line(frame, (int(panel_x + 25 + threshold_pos), int(y_bar - 2)), 
                    (int(panel_x + 25 + threshold_pos), int(y_bar + conf_bar_height + 2)), 
                    self.colors['text_primary'], 2)
            
            cv2.putText(frame, f"Req: {required_threshold:.2f}", (int(panel_x + 25), int(y_pos + 105)), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.4, self.colors['text_muted'], 1)
        else:
            cv2.putText(frame, "No prediction", (int(panel_x + 25), int(y_pos + 50)), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, self.colors['text_muted'], 1)
        
        y_pos += prediction_card_height + 20
        
        # Last Accepted Word
        if self.last_accepted_word:
            cv2.putText(frame, f"Last: {self.last_accepted_word}", (int(panel_x + 25), int(y_pos)), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, self.colors['accent'], 1)
        
        y_pos += 25
        
        # System Info Section
        system_card_height = 60
        self.draw_rounded_rect(frame, (panel_x + 15, y_pos), 
                              (panel_x + panel_width - 15, y_pos + system_card_height), 
                              self.colors['bg_light'], -1, 8)
        
        cv2.putText(frame, f"Threshold: {self.confidence_threshold:.1f}", (int(panel_x + 25), int(y_pos + 20)), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, self.colors['text_secondary'], 1)
        
        # TTS Status
        tts_status = "Google TTS" if self.tts_available else "Unavailable"
        tts_color = self.colors['success'] if self.tts_available else self.colors['error']
        cv2.putText(frame, f"TTS: {tts_status}", (int(panel_x + 25), int(y_pos + 40)), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, tts_color, 1)
        
        y_pos += system_card_height + 30
        
        # Modern Controls Section
        cv2.putText(frame, "CONTROLS", (int(panel_x + 25), int(y_pos)), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, self.colors['text_primary'], 2)
        y_pos += 20
        
        controls = [
            ("R", "Record", "primary"),
            ("C", "Clear", "warning"),
            ("U", "Undo", "warning"),
            ("A", "Auto-Speech", "success" if self.auto_speech else "default"),
            ("S", "Speak", "accent"),
            ("Q", "Quit", "error")
        ]
        
        # Draw control buttons with dynamic sizing
        button_height = 30
        buttons_per_row = 3
        button_spacing = 8
        
        # Calculate button positions and sizes
        current_x = panel_x + 25
        current_y = y_pos
        buttons_in_row = 0
        
        for i, (key, label, color_scheme) in enumerate(controls):
            # Create button text and calculate size
            button_text = f"{key}: {label}"
            text_size = cv2.getTextSize(button_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]
            button_width = text_size[0] + 20  # Add padding
            
            # Check if we need to move to next row
            if buttons_in_row >= buttons_per_row:
                current_y += button_height + button_spacing
                current_x = panel_x + 25
                buttons_in_row = 0
            
            # Special highlighting for active states
            is_active = False
            if key == "R" and self.is_recording:
                is_active = True
                color_scheme = "success"
            elif key == "A" and self.auto_speech:
                is_active = True
            
            self.draw_modern_button(frame, current_x, current_y, button_width, button_height, 
                                  button_text, is_active, color_scheme)
            
            current_x += button_width + button_spacing
            buttons_in_row += 1
        
        return frame
    
    def draw_enhanced_transcription(self, frame, width, height):
        """Draw the transcription panel (no animations)"""
        if not self.detected_words:
            return frame
        
        # Modern transcription area
        trans_height = 140
        trans_y = height - trans_height
        panel_width = width - 450  # Exclude info panel with margin
        
        # Modern background
        overlay = frame.copy()
        self.draw_rounded_rect(overlay, (20, trans_y), (panel_width, height - 20), 
                              self.colors['bg_dark'], -1, 12)
        frame = cv2.addWeighted(frame, 0.3, overlay, 0.7, 0)
        
        # Modern border with accent color
        self.draw_rounded_rect(frame, (20, trans_y), (panel_width, height - 20), 
                              self.colors['primary'], 3, 12)
        
        # Header
        header_height = 35
        self.draw_rounded_rect(frame, (25, trans_y + 5), (panel_width - 5, trans_y + header_height), 
                              self.colors['bg_medium'], -1, 8)
        
        cv2.putText(frame, "TRANSCRIPTION", (35, trans_y + 25), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, self.colors['text_primary'], 2)
        
        # Word count badge
        word_count = len(self.detected_words)
        badge_text = f"{word_count} words"
        badge_size = cv2.getTextSize(badge_text, cv2.FONT_HERSHEY_SIMPLEX, 0.4, 1)[0]
        badge_x = panel_width - badge_size[0] - 40
        
        self.draw_rounded_rect(frame, (badge_x - 5, trans_y + 10), 
                              (badge_x + badge_size[0] + 5, trans_y + 25), 
                              self.colors['accent'], -1, 5)
        cv2.putText(frame, badge_text, (badge_x, trans_y + 20), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, self.colors['text_primary'], 1)
        
        # Transcription text area
        text_area_y = trans_y + header_height + 10
        text_area_height = trans_height - header_height - 25
        
        # Create text background
        self.draw_rounded_rect(frame, (25, text_area_y), (panel_width - 5, trans_y + trans_height - 15), 
                              self.colors['bg_light'], -1, 8)
        
        # Simple text rendering (no highlighting animations)
        sentence = " ".join(self.detected_words[-20:])  # Show last 20 words
        max_width = panel_width - 60
        font_scale = 0.7
        font_thickness = 1
        
        # Word wrapping
        words = sentence.split()
        lines = []
        current_line = ""
        
        for word in words:
            test_line = current_line + " " + word if current_line else word
            text_size = cv2.getTextSize(test_line, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)[0]
            
            if text_size[0] < max_width:
                current_line = test_line
            else:
                if current_line:
                    lines.append(current_line)
                    current_line = word
                else:
                    lines.append(word)
        
        if current_line:
            lines.append(current_line)
        
        # Draw lines (max 3 lines, no animations)
        lines = lines[-3:]
        line_spacing = 25
        
        for i, line in enumerate(lines):
            y_text = text_area_y + 25 + (i * line_spacing)
            
            # Simple color scheme without fade effects
            if i == len(lines) - 1:  # Last line
                text_color = self.colors['text_primary']
            elif i == len(lines) - 2:  # Second to last line
                text_color = self.colors['text_secondary']
            else:  # Older lines
                text_color = self.colors['text_muted']
            
            cv2.putText(frame, line, (35, y_text), cv2.FONT_HERSHEY_SIMPLEX, 
                       font_scale, text_color, font_thickness)
        
        return frame
    
    def draw_interface(self, frame):
        """Draw the complete enhanced professional interface (no notifications)"""
        height, width = frame.shape[:2]
        
        # Draw enhanced info panel
        frame = self.draw_enhanced_info_panel(frame, width, height)
        
        # Draw enhanced transcription
        frame = self.draw_enhanced_transcription(frame, width, height)
        
        return frame
    
    def run(self):
        """Main inference loop with enhanced professional interface"""
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            print("Error: Could not open camera")
            return
        
        # Set camera properties for optimal performance
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        self.cap.set(cv2.CAP_PROP_FPS, 30)
        
        print("SIBI Professional Real-time Inference System")
        print("=" * 60)
        print("Enhanced Professional UI Loaded")
        print("Modern Control Interface Active")
        print("Professional TTS System Ready")
        print("High-Performance Video Processing")
        print("=" * 60)
        
        print("\nProfessional Controls:")
        print("  'R' - Toggle Recording (Start/Stop)")
        print("  'C' - Clear Transcription")
        print("  'U' - Undo Last Word")
        print("  'A' - Toggle Auto-Speech")
        print("  'S' - Manual Speech")
        print("  'Q' - Professional Exit")
        
        print(f"\nSystem Configuration:")
        print(f"  Confidence Threshold: {self.confidence_threshold}")
        print(f"  Auto-Speech: {self.auto_speech}")
        print(f"  TTS System: {'Google TTS (Professional)' if self.tts_available else 'Disabled'}")
        if self.class_specific_thresholds:
            print("  Custom Thresholds:")
            for cls, threshold in self.class_specific_thresholds.items():
                print(f"    {cls}: {threshold}")
        
        # Performance measurement
        print(f"\nMeasuring System Performance...")
        
        fps_samples = []
        measurement_start = time.time()
        measurement_duration = 2.0
        
        while time.time() - measurement_start < measurement_duration:
            ret, frame = self.cap.read()
            if ret:
                self.update_fps()
                if self.current_fps > 0:
                    fps_samples.append(self.current_fps)
                
                frame = cv2.flip(frame, 1)
                
                # Professional loading screen
                overlay = frame.copy()
                self.draw_rounded_rect(overlay, (0, 0), (frame.shape[1], frame.shape[0]), 
                                      self.colors['bg_dark'], -1, 0)
                frame = cv2.addWeighted(frame, 0.2, overlay, 0.8, 0)
                
                # Loading without animations
                remaining = measurement_duration - (time.time() - measurement_start)
                progress = 1 - (remaining / measurement_duration)
                
                # Center loading elements
                center_x = frame.shape[1] // 2
                center_y = frame.shape[0] // 2
                
                cv2.putText(frame, "SIBI PROFESSIONAL", (int(center_x - 150), int(center_y - 60)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1.2, self.colors['primary'], 3)
                cv2.putText(frame, "Initializing System...", (int(center_x - 100), int(center_y - 20)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, self.colors['text_secondary'], 2)
                
                # Simple loading bar
                bar_width = 300
                bar_height = 8
                bar_x = center_x - bar_width // 2
                bar_y = center_y + 10
                
                self.draw_progress_bar(frame, bar_x, bar_y, bar_width, bar_height, 
                                      progress, self.colors['bg_medium'], self.colors['primary'], 4)
                
                cv2.putText(frame, f"Performance: {self.current_fps:.1f} FPS", 
                           (int(center_x - 80), int(center_y + 40)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, self.colors['success'], 2)
                
                cv2.imshow('SIBI Professional', frame)
                cv2.waitKey(1)
        
        if fps_samples:
            avg_fps = sum(fps_samples) / len(fps_samples)
            performance_status = "Excellent" if avg_fps > 25 else "Good" if avg_fps > 20 else "Adequate"
            print(f"System Performance: {performance_status}")
            print(f"   Average FPS: {avg_fps:.1f}")
            print(f"   Performance Rating: {performance_status}")
        
        print(f"\nProfessional Interface Active!")
        print("Press 'R' to begin sign language detection")
        
        try:
            while True:
                ret, frame = self.cap.read()
                if not ret:
                    print("Error: Could not read frame")
                    break
                
                self.update_fps()
                frame = cv2.flip(frame, 1)
                
                if self.is_recording:
                    self.process_frame(frame)
                
                # Apply professional interface
                display_frame = self.draw_interface(frame)
                
                cv2.imshow('SIBI Professional', display_frame)
                
                # Enhanced key handling
                key = cv2.waitKey(1) & 0xFF
                if key == ord('q') or key == ord('Q'):
                    break
                elif key == ord('r') or key == ord('R'):
                    if self.is_recording:
                        self.stop_recording_session()
                    else:
                        self.start_recording_session()
                elif key == ord('c') or key == ord('C'):
                    self.detected_words.clear()
                    self.current_prediction = ""
                    self.current_confidence = 0.0
                    self.last_accepted_word = None
                    print("Transcription cleared")
                elif key == ord('u') or key == ord('U'):
                    if self.detected_words:
                        removed_word = self.detected_words.pop()
                        self.last_accepted_word = self.detected_words[-1] if self.detected_words else None
                        print(f"Removed: {removed_word}")
                elif key == ord('a') or key == ord('A'):
                    self.auto_speech = not self.auto_speech
                    status = "enabled" if self.auto_speech else "disabled"
                    print(f"Auto-speech {status}")
                elif key == ord('s') or key == ord('S'):
                    if self.detected_words:
                        sentence = " ".join(self.detected_words)
                        self.speak_text(sentence, immediate=True)
                    else:
                        print("No text to speak")
                
        except KeyboardInterrupt:
            print("\nProfessional shutdown initiated")
        finally:
            # Professional shutdown
            print(f"\nProfessional Session Summary:")
            fps_stats = self.get_overall_fps_stats()
            if fps_stats:
                print(f"   Average Performance: {fps_stats['average']:.1f} FPS")
                print(f"   Peak Performance: {fps_stats['max']:.1f} FPS")
                print(f"   Total Frames: {fps_stats['total_frames']:,}")
            
            if self.detected_words:
                print(f"   Words Detected: {len(self.detected_words)}")
                print(f"   Final Transcription: {' '.join(self.detected_words)}")
            
            # Clean shutdown
            if self.tts_available:
                self.tts_queue.put(None)
                self.tts_thread.join(timeout=2)
            
            self.cap.release()
            cv2.destroyAllWindows()
            if self.tts_available:
                pygame.mixer.quit()
            
            print("SIBI Professional System Shutdown Complete")

# Professional Usage Example
if __name__ == "__main__":
    # SIBI class definitions
    classes = [
        "Anda", "Apa", "Bagaimana", "Berapa", "Buku", "Dia", "Dosen", 
        "Kami", "Kapan", "Kelas", "Kelompok", "Kursi", "Mengapa", 
        "Mereka", "Nilai", "Proyek", "Saya", "Siapa", "Tugas", "Tulis"
    ]
    
    print("SIBI PROFESSIONAL DETECTION SYSTEM")
    print("=" * 50)
    print(f"Loaded {len(classes)} Professional Sign Classes")
    for i, cls in enumerate(classes):
        print(f"  {i:2d}: {cls}")
    print("=" * 50)
    
    model_path = "base_weights_dataset1.pth"
    
    try:
        # Professional configuration
        class_thresholds = {
            "Apa": 0.9  # Higher threshold for problematic classes
        }
        
        # Initialize Professional System
        professional_system = RealTimeSIBIInference(
            model_path=model_path,
            class_names=classes,
            device='cuda' if torch.cuda.is_available() else 'cpu',
            sequence_length=30,
            confidence_threshold=0.8,
            cooldown_frames=30,
            class_specific_thresholds=class_thresholds,
            auto_speech=True
        )
        
        professional_system.run()
        
    except FileNotFoundError as e:
        print(f"Model Error: {e}")
        print("Please verify model file path and existence.")
    except Exception as e:
        print(f"System Error: {e}")
        import traceback
        traceback.print_exc()